In [1]:
import pandas as pd
import os
import numpy as np

# Dataset

In [3]:
data_file = r"C:\Users\jeuux\Desktop\Carrera\MoAI\TFM\AnnotatedData\FinalDatasets\Participants\1105e\FullDataset\Final_data_1105e.pkl"
data = pd.read_pickle(data_file)

In [4]:
data

,GazeX,GazeY,GazeZ,Recording timestamp,target,picture,AccX,AccY,AccZ,GyroX,GyroY,GyroZ
0,152.0096,20.2408,497.4997,20,AG Null,Null,0.674664,-9.787647,2.612879,10.770769,106.059807,0.904581
1,91.5267,-15.7177,365.1143,40,AG Null,Null,0.385073,-9.946392,1.999057,15.626822,133.458525,1.011099
2,189.4057,-9.6084,886.4117,60,AG Null,Null,-0.979491,-9.978140,2.121815,12.625428,110.787960,-7.364873
3,170.1867,-26.8524,509.0201,80,AG Null,Null,-0.732546,-9.611419,2.078551,16.008721,116.409430,-34.749907
4,169.8932,-27.7286,369.5690,100,AG Null,Null,0.333401,-9.609714,1.900401,20.392657,98.363320,-45.158527
...,...,...,...,...,...,...,...,...,...,...,...,...
58293,-35.7709,288.0234,1090.1591,1253432,AG Null,Null,-1.192481,-10.129268,-1.133554,28.868055,107.195244,-7.847105
58294,-58.2455,247.9092,1099.1627,1253452,AG Null,Null,-1.218537,-10.214132,-1.161397,35.098475,101.830436,-2.509971
58295,-19.8429,1.7944,273.0716,1253472,AG Null,Null,-1.441757,-10.147727,-1.084285,37.155527,102.776885,10.210608
58296,-69.3203,-23.4384,262.0244,1253492,AG Null,Null,-1.544585,-10.011470,-1.112609,34.284361,98.902668,13.367851


In [5]:
def windows(data, size,factor=2):
    start = 0
    while start + (size / factor) < len(data):
        yield int(start), int(start + size)
        start += (size / factor)


In [6]:
def to_seconds(time,factor = 1000):
    seconds = time/factor
    return round(seconds,2)

In [7]:
def get_pictures_dataset(data,user,window_size,factor =1):
    df_pictures = {"id":[],"target":[],"start":[],"end":[]}
 
    for idx,(start, end) in enumerate(windows(data.index, window_size,factor)):
        df_pictures["id"].append(f"{user}_{idx}")
        df_pictures["target"].append(data.loc[start:end,"picture"].mode()[0])
        df_pictures["start"].append(to_seconds(data.loc[start,"Recording timestamp"]))
        df_pictures["end"].append(to_seconds(data.loc[end,"Recording timestamp"]))
        
    return pd.DataFrame.from_dict(df_pictures).set_index("id")

In [8]:
user = "1105e"
test = get_pictures_dataset(data,user,500)

In [9]:
def filter_nulls(df):
    #get samples of majoritary class different than null
    samples_max=max(test[test["target"]!="Null"]
                    .target \
                    .value_counts() ) 
    #get id for null and rest of labels
    null_idx=df[df["target"]=="Null"].index
    samples_null = len(null_idx)
    if(samples_null>samples_max):
        #perform random undersampling
        rest_idx = df[df["target"]!="Null"].index
        # sample a subset of id from null samples to match majoritary class samples
        selected_idx = np.random.choice(range(samples_null), size=samples_max, replace=False)
        null_idx=null_idx[selected_idx]
        #filter df 
        df = df.loc[null_idx|rest_idx]

    return df
    

In [10]:
df= filter_nulls(test)

In [11]:
df

,target,start,end
id,,,
1105e_100,_Y3A6113,1063.60,1073.96
1105e_101,_Y3A6113,1073.96,1084.25
1105e_102,Null,1084.25,1094.71
1105e_11,_Y3A6125,118.23,131.36
1105e_114,Null,1223.19,1235.14
1105e_16,_Y3A6096,174.14,184.51
1105e_22,_Y3A6093,239.74,249.98
1105e_24,_Y3A6092,261.07,271.16
1105e_25,_Y3A6092,271.16,281.16


# Video

In [12]:
from moviepy.editor import *

In [13]:
path_video = r"C:\Users\jeuux\Desktop\Carrera\MoAI\TFM\AnnotatedData\Videos_raw\1105e.mp4"
video = VideoFileClip(path_video).resize((224,224))

In [14]:
df = df.reset_index()

In [29]:
video_dataset = {
    "video_path": [],
    "label": [],
    "frames": []
}
root_path = r"C:\Users\jeuux\Desktop\Carrera\MoAI\TFM\AnnotatedData\FinalDatasets\Participants"
video_folder = r"C:\Users\jeuux\Desktop\Carrera\MoAI\TFM\AnnotatedData\FinalDatasets\Participants\1105e\Clips"
participant ="1105e"
if not(os.path.isdir(video_folder)):
    os.makedirs(video_folder)

In [30]:
for idx in tqdm(df.index):
    #get clip
    clip = video.subclip(df.loc[idx,"start"],df.loc[idx,"end"])
    n_frames  = clip.duration * clip.fps
    label  = df.loc[idx,"target"]
    # Write the result to a file 
    clip_id = df.loc[idx,"id"]
    video_rel_path = os.path.join(participant,"Clips",f"{clip_id}.mp4")
    clip_file= os.path.join(root_path,video_rel_path)
    clip.write_videofile(clip_file,audio=False,logger = None)
    #update dataset
    video_dataset["video_path"].append(video_rel_path)
    video_dataset["label"].append(label)
    video_dataset["frames"].append(n_frames)
    

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [10:01<00:00, 18.78s/it]


In [31]:
video_dataset

{'video_path': ['1105e\\Clips\\1105e_100.mp4',
  '1105e\\Clips\\1105e_101.mp4',
  '1105e\\Clips\\1105e_102.mp4',
  '1105e\\Clips\\1105e_11.mp4',
  '1105e\\Clips\\1105e_114.mp4',
  '1105e\\Clips\\1105e_16.mp4',
  '1105e\\Clips\\1105e_22.mp4',
  '1105e\\Clips\\1105e_24.mp4',
  '1105e\\Clips\\1105e_25.mp4',
  '1105e\\Clips\\1105e_26.mp4',
  '1105e\\Clips\\1105e_29.mp4',
  '1105e\\Clips\\1105e_3.mp4',
  '1105e\\Clips\\1105e_4.mp4',
  '1105e\\Clips\\1105e_42.mp4',
  '1105e\\Clips\\1105e_52.mp4',
  '1105e\\Clips\\1105e_55.mp4',
  '1105e\\Clips\\1105e_63.mp4',
  '1105e\\Clips\\1105e_64.mp4',
  '1105e\\Clips\\1105e_65.mp4',
  '1105e\\Clips\\1105e_66.mp4',
  '1105e\\Clips\\1105e_68.mp4',
  '1105e\\Clips\\1105e_69.mp4',
  '1105e\\Clips\\1105e_70.mp4',
  '1105e\\Clips\\1105e_72.mp4',
  '1105e\\Clips\\1105e_73.mp4',
  '1105e\\Clips\\1105e_74.mp4',
  '1105e\\Clips\\1105e_75.mp4',
  '1105e\\Clips\\1105e_76.mp4',
  '1105e\\Clips\\1105e_78.mp4',
  '1105e\\Clips\\1105e_8.mp4',
  '1105e\\Clips\\1105e_9.